<style>
    .div-1 {
        background-color: #EBEBEB;
    }
    
    .div-2 {
    	background-color: #ABBAEA;
    }
    
    .div-3 {
    	background-color: #FBD603;
    }
</style>

<div align="right"><img src='https://blockchain4talent.com/wp-content/uploads/2021/09/a58ab6e0efdce2743f592019683b00c2.jpeg'>

<h1 style="color:#2596be;font:luminary;text-align:center;"><i>Cisco Project</i></h1>

<div class=“alert alert-success”>

<h2 style="color:#2596be;">Project's Description</h1> 


This team will create a __Time Series Analysis__ model that will predict the sales for Cisco a year in the future. Our goal is to explore historical cisco sales to Texas public agencies (school districts, government agencies, municipalities, etc) to present to people who might want to invest into the cisco ecosystem. This can include potential stock investors or customers or even people looking to train to properly implement Cisco equipment.



In [5]:
# project modules
import wrangle as wr
import explore_Copy1 as ex
import summaries as s
#import ts_models_d as tsmd

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
# makes vizualizations better in a jupyter notebook
%config InlineBackend.figure_format = 'retina'

ModuleNotFoundError: No module named 'src'

<h3 style="color:#2596be;">Acquire and Prepare</h3> 

We acquired the data from the [Texas Open Data Portal](https://data.texas.gov/dataset/OFFICIAL-DIR-Cooperative-Contract-Sales-Data-Fisca/w64c-ndf7). The origin of the data is 	[Texas Department of Information Resources](	https://dir.texas.gov/). The data file contains information about DIR (Department of Information Resources) Cooperative Contract Sales. To get the information about Cisco we filtered the data by vendor name.

- Our first cleaning step was dropping the  7 columns with null values only and renaming columns into programming friendly format simply by replacing whitespaces with underscores and making them lower case. Our next step was dropping columns with no value for exploration or modeling purposes (for example, `vendor_adress`, `reseller_phone` etc.)
- We have created additional features based on the `order_date` information:
    `year` : year
    `quarter`: quarter
    `month`: month number
    `month_name`: month name
    `day_of_week`: day of week number
    `day_name`: day of week name
    `day_of_year`: day of the year
    `week`: week of the year number

The original data contains the accounting information about every transaction where every row represents a transaction. One company could have many transactions per day, including those where the `purchase_amount` was equal to zero, one cent or being negative. We have created as well a summary data frame whith the final `purchase_amount` per day per company. This combined the number of rows from 261,886 to 34,401 rows. 
- Outliers were kept for this iteration of explore of modeling.

- To avoid data leakagee divided the data into train and test sets. 
- After the exploration we split the test set into validation and test sets as well.
- __The target variable of the project is ```purchase_amount```__


<h3 style="color:#2596be;">Data Exploration</h3> 

Before we jump into Data Exploration, let's have a look at the customer types represented in our data. We can see that most orders are made by Local Goverments, followed by  Independent School Districts, Higher Education Institutions and State Agencies. Other public agencies make only 0.6% orders.

In [ ]:
# call pie chart with 
ex.viz_customer_types()

### 1. Is there any significant difference in average of total purchase amounts by  day?

In [ ]:
# visualize average sales per day
ex.q1_viz()

We can see a significant difference in purchase amount between weekdays and weekends. The daily  average  of total purchase amount is plotted as a horizontal purple dashed line. Based on this graph we want to run two statistical tests. We will check if the average of total purchase amount by day is different from average of weekly purchase amounts. Next we will look into the averages of total purchase amount of Monday through Wednesday and see if there is a difference in their averages.

For statistical tests:

- $H_0:$ stands for 'Null hypothesis’ 
- $H_a:$ stands for 'Alternative hypotheses'

- All statistical tests run for the **95% Confidence level**.
- The **significance level** (alpha) is set to **0.05**
- If the p-value is smaller than 0.05 we reject the Null hypothesis.

#### Test #1

$H_0$: There is no significant difference in average total purchase amount by day  and average of weekly purchase amounts.

$H_a$: There is a significant difference in average total purchase amount by day  and average of weekly purchase amounts.

We use 1 sample T-test to compare average of weekly purchase amounts of an independent sample against the population average of weekly purchase amounts.

In [ ]:
# call ttest for every day of the week
ex.q1_ttest()

All days showed a significant difference. For every day of the week we reject the null hypothesis.

#### Test #2

$H_0$: There is no statisticaly significant difference in means of sales during work days Mon through Wed


$H_a$: There is a statisticaly significant difference in means of sales during work days Mon through Wed

In [ ]:
# call ANOVA test
ex.q1_anova()

The statistical test showed that average sales on Monday, Tuesday and Wednesday don't have significant difference.

#### Takeaways:
- All daily purchase amounts are significantly different from the average of weekly purchase amounts 
- Week days have higher average purchase amounts compared to average of weekly purchase amounts
- Weekend days purchase amounts are well below average of weekly purchase amounts
- Monday, Tuesaday and Wednesday seem to have same average amount of sales
- Friday has the highest average sales amount among all days of the week.

### 2. Is there any significant difference in average of total purchase amount by month?

In [ ]:
# show average sales per month
ex.q2_viz()

It seems that July has the largest average purchase amount and February has the lowest average purchase amount.

__One sample T-test to check if the average sales per month differ from average monthly sales.__

$H_0$: There is no significant difference in  average of total purchase  per month and average yearly purchase amount.

$H_a$: There is a significant difference in  average of total purchase  per month and average yearly purchase amount.

In [ ]:
# run statistical test for every month
ex.q2_ttest()

#### Takaways:
- The highest purchase amount are happening in July, followed by April, October and June.
- The lowest sales happen on February.
- March has almost the same results as average sales, but there was peak in sales on March, 2020 when pandemic just started, it might happened that March is typically low on sales.
- April results might be affected by beginning of pandemic, too.

### 3. Is there any significant difference in purchase amount  by the quarter?

In [ ]:
# visualize quarters
ex.q3_viz()

Looks like first and third quarter are significantly different from the quarterly mean of purchase amount. In the first quarter purchase amounts are much lower and in the third quarter they are higher. Let's check this observation with a statistical test. We'll run a sample T-test to compare mean of the sample against the mean of the population.

$H_0$: The quarterly purchase amount of each quarter are not significantly different from overall mean quarterly purchase amount.

$H_a$: The quarterly purchased amount of each quarter are significantly different from overall quarterly purchase amount.

In [ ]:
# run t-test
ex.q3_ttest()

Despite the obvious difference in the quarterly purchase amount on the bar chart, statistical test disproved this statement. We fail to reject the null hypothesis because the difference in means is not significant.

#### Takeaways:
- Highest sales are in the 3rd quarter.
- Lowest sales are in the 1st quarter.
- Average qurterly sales per each quarter are not significantly different from the overall average quarterly sales.

### 4. Is there any significant change in monthly purchase amount percentage in our data?

Now let's have a closer look into monthly percentage change in total sales. The graph below shows clear spike in total sales in March, 2020, exactly in the beginning of COVID-2019 pandemic.

In [ ]:
# show monthly % change
ex.q4_viz()

### Takeaway:
In the beginning of the pandemic there was a spike in sales.

### 5. Were there any abnormal purchase orders during the time frame in the data set, if so by who?
- We know that the covid-19 global pandemic altered business in the United States among countries of the world. Certainly Cisco Systems' purchase amount in the state of Texas would be effected somehow. We set out to see if there would be large anamolous activities around the year 2020. First, we found that there is an enormous jump in quantities of orders from customers. This jump occured on Janurary 2, 2020. Texas Health and Human Services Commission purchased 3,767,123 units of something and 865,822 units of something else. 

- We found that a large purchase order like this is made for Texas Health and Human Services periodically, but the purchase order is limited to around 1 million. This purchase probably had to do with foresight regarding what could come given the brewing pandemic which was being reported in China.

- You may follow the following graphs to see the findings

__Notice the large order quantity outlier__

Texas Health and Human Services orders.

In [ ]:
# show the orders and payments history for THHS
ex.q5_thhsc()

In [ ]:
# show the orders on Jan, 2 2020
ex.q5_vizC()

We can see that Texas Health and Human Services Commission purchased close to 4 million somethings from Cisco Systems one time, outside of any predictable pattern

<h3 style="color:#2596be;">Data Exploration Takeaways</h3> 

- Cisco customers represented in DIR data set are divided into following groups:
    * Local Governments - 34.82%
    * Independent School Districts - 31.80 %
    * Higher Education Institutions - 22.54%
    * State Agencies - 10.27%
    * Other public agencies - 0.56%
    
__Daily purchase amounts__
- All purchase amounts per day of the week were significantly different from the average weekly purchase amounts.
- Week days have higher sales than the average and weekend days almost don't have sales.
- Monday, Tuesaday and Wednesday seem to have same average amount of purchase amounts.
- Friday has the highest average purchase amount among all days of the week.

__Monthly purchase amounts__
- The highest purchase amounts are happening in July, followed by April, October and June.
- The lowest purchase amounts happen on February.
- March has almost the same results as average purchase amounts, but there was peak in purchase amounts on March, 2020 when pandemic just started, it might happened that March is typically low on purchase amounts.
- April results might be affected by beginning of pandemic, too.

__Quarterly purchase amounts__
- Highest purchase amounts are in the 3rd quarter.
- Lowest purchase amounts are in the 1st quarter.
- Average qurterly purchase amounts per each quarter are not significantly different from the overall average quarterly purchase amounts.

__Pandemic observations__
- In the beginning of the pandemic there was a spike in purchase amounts.

<h3 style="color:#2596be;">Time Series Modeling</h3> 

To prepare the data set for time series modeling we created the data frame that contains the information only about the sale's date and amount. The `purchase_amount` is the total sale number by the end of the day. At the removed all information about customers and customer types.

In [ ]:
from importlib import reload

In [ ]:
reload(tsmd)

In [ ]:
# show first 3 rows of our train data
tsmd.X_train.head(3)

We left the train set the same. It includes data till the end of 2021. We split our test set into validate and test sets. The validate set has data January till June, 2022. The test set contains information about sales in time period July - November, 2022.

Let's have a quick look how sales are changing through the time.

In [ ]:
# show daily sales visualization
tsmd.show_ts()

This graph doesn't give us much hope. We data looks pretty noisy. We can not observe any trend or seasonality that simplify time series predictions. The sales spikes are inconsistent.

### Data evaluation

To evaluate the model performance we going to use __RMSE score__. This score calculates the absolute error. The lower is RMSE score, the better is model performance. The RMSE score should not be higher than the baseline's model score. __The baseline model__ is the model where all values are the same. In our case it is the average daily sale amount. We going to evaluate our model on the validate set to see if it has good predicting power.

In [ ]:
# show the baseline value
tsmd.baseline

In [ ]:
# evaluate and plot baseline
tsmd.evaluate()

Because we don't observe any trend in our data we can not use Holt's Linear Trend models. We decided to try ARIMA models, that rely on moving average and stationarity of the data. The results of ARIMA models you can see below.

In [ ]:
# create ARIMA models and show their scores
tsmd.create_arima_models()

The first three models beat baseline score but just by a little bit, and they definately are not good.

The next step we took is building regression model with XGBoost Regressor. 

In [ ]:
# create XGBoost model
tsmd.run_xgboost()

XGBosst Regressor model beats the baseline's RMSE score and outperformes all other models that we've created. We are going to use XGBoost Regressor for our test data set.

In [ ]:
# show scores and predictions of the test set
tsmd.run_test_model()

#### Modeling summary

- Our model beats a baseline on all train, validation and test sets. Anyway, there is much more work to do. We'd like to improve results by making weekly/monthly resamples hoping find some seasonality. We'd like to remove the some pandemic sales anomalies as well. If this won't improve model performance, our next step is going to be splitting data on 5 different sets based on the customer type and make predictions for every set.

<h3 style="color:#2596be;">Conclusions and next steps</h3> 

The goals of this project were:
- Explore the historical data of the DIR contracts with Cisco. 
- Create a model for the sales forecast of Cisco with public agencies.

Our model performs better than a baseline model but needs some improvents.

As the next step for this project we'd like to:
- create weekly/monthly resamples hoping find some seasonality and trends
- remove the some pandemic sales anomalies as well

or, as final step
- split data on 5 different sets based on the customer type and mae predictions for every set.